In [1]:
from sys import path
from dolfin import *
from mshr import *
from Python_module_Quang import *

import matplotlib.pyplot as plt
import sys
from numpy.core.records import array

from numpy.lib.function_base import append
sys.path.append(
    '/media/xuanquang/Gaumap Lab data/05_Git_project/FEniCS-RBniCS-examples/20210218_2D_tangential_load/'
)

try:
    import file
except:
    print("fail to import file")

try:
    import tangential_load
except:
    print("fail to import tangential_load.py")

format = "png"